<a href="https://colab.research.google.com/github/JamieCDev20/GAM705/blob/main/705Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

Imports

In [54]:
import numpy as np
!pip install pyquaternion
from pyquaternion import Quaternion

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


File Reader

In [81]:
class AnimFileReader():
  ## "#" breaks between nodes | ":" breaks between values

  def __init__(self, _path):
    self.path = _path
    self.animData = np.array(float)
    self.__ReadAnim__(self.path)

  def __ReadAnim__(self, path):
    file = open(path, 'r')
    overallData = file.read()
    overallData = overallData.split("SS")
    del overallData[0]

    rots = overallData[0].split('#')[0].split(':')
    poss = overallData[0].split('#')[1].split(':')
    
    del rots[0]
    del poss[0]

    temp = rots[1]
    temp = temp.replace('(', '')
    temp = temp.replace(')', '')

    temp = temp.split(',')

    a = Quaternion(temp[0], temp[1], temp[2],temp[3])
    
    temp = rots[2]
    temp = temp.replace('(', '')
    temp = temp.replace(')', '')

    temp = temp.split(',')

    b = Quaternion(temp[0], temp[1], temp[2],temp[3])

    print(a)
    print(b)
    print(a*(a/b)) ## <<< (a/b) gives the quaternion that would rotate a to b

    print(len(rots))
    print(rots)

   

In [82]:
fr = AnimFileReader("SHO_EXORCISMWALK_1_UNCLEAN.txt")

-0.140 -0.038i +0.008j +0.989k
-0.140 -0.038i +0.007j +0.989k
-0.140 -0.038i +0.007j +0.989k
221
['(0.000000,0.000000,0.000000,1.000000)', '(-0.140425,-0.038209,0.007743,0.989323)', '(-0.140410,-0.038145,0.007387,0.989331)', '(-0.139038,-0.037941,0.007163,0.989534)', '(-0.139616,-0.038612,0.006649,0.989430)', '(-0.142235,-0.039313,0.006135,0.989033)', '(-0.146261,-0.039890,0.005730,0.988425)', '(-0.148566,-0.040552,0.005683,0.988054)', '(-0.150725,-0.040799,0.005528,0.987718)', '(-0.153034,-0.040649,0.005191,0.987371)', '(-0.155468,-0.040551,0.004814,0.986997)', '(-0.158231,-0.041064,0.004699,0.986537)', '(-0.161332,-0.042077,0.004572,0.985992)', '(-0.164681,-0.042875,0.003066,0.985410)', '(-0.167946,-0.041206,0.000218,0.984935)', '(-0.170285,-0.037841,-0.002701,0.984664)', '(-0.169389,-0.038438,-0.000243,0.984799)', '(-0.162556,-0.040420,0.002332,0.985868)', '(-0.149518,-0.042821,0.003853,0.987824)', '(-0.131318,-0.043789,0.004786,0.990361)', '(-0.114669,-0.044238,0.004975,0.992406)',

Testing

In [51]:
a = np.array([1, 2, 3, 4, 5], float)
b = np.array([6, 6, 6, 6, 6], float)
c = b-a
c = np.delete(c, 0)
print(c)


[4. 3. 2. 1.]
